# Adjacency matrix


In [1]:
# !pip install iduedu

In [2]:
# To read .parquet
# !pip install pyarrow

In [3]:
import geopandas as gpd

# Read a GeoDataFrame, for example this one
data = gpd.read_parquet('data/spb_buildings.parquet')

# Create a polygon that encircles all the buildings in the dataset to download the graph
polygon = data.to_crs(4326).union_all().convex_hull.buffer(0.001)

In [7]:
from iduedu import get_intermodal_graph, get_drive_graph, get_walk_graph

# Download the intermodal graph
G_intermodal = get_intermodal_graph(territory=polygon)

# Uncomment the following lines to download walking or driving graphs for the same area
# G_walk = get_walk_graph(polygon =polygon)
# G_drive = get_drive_graph(polygon =polygon)

2025-10-07 17:52:59.701 | INFO     | Downloading walk network via Overpass ...


2025-10-07 17:53:11.770 | WARNING  | Graph contains 58 weakly connected components. This means the graph has disconnected groups if edge directions are ignored. Component sizes:: [4235, 19, 9, 7, 5, …]
2025-10-07 17:53:11.782 | WARNING  | Removing 164 nodes from 57 smaller strongly connected components. These are subgraphs where nodes are internally reachable but isolated from the rest. Retaining only the largest strongly connected component (4235 nodes).


Parsing public transport routes:   0%|          | 0/53 [00:00<?, ?it/s]

2025-10-07 17:54:32.280 | INFO     | Composing intermodal graph...
2025-10-07 17:54:33.021 | WARNING  | Removing 746 nodes from 359 smaller strongly connected components. These are subgraphs where nodes are internally reachable but isolated from the rest. Retaining only the largest strongly connected component (6062 nodes).


In [5]:
import numpy as np
from iduedu import get_adj_matrix_gdf_to_gdf

# Compute the adjacency matrix between the objects in the dataset using the intermodal graph.
# The weight of the edges is set to 'time_min' (travel time in minutes), and the data type is set to np.float32 for precision.
matrix = get_adj_matrix_gdf_to_gdf(data, data, G_intermodal, weight='time_min', dtype=np.float32)

# Uncomment the following lines to compute adjacency matrices using driving or walking graphs with different weights and data types
# matrix = get_adj_matrix_gdf_to_gdf(data,data,G_drive,weight='time_min',dtype=np.float16) # Use dtype >np.float16 for more precision
# matrix = get_adj_matrix_gdf_to_gdf(data,data,G_walk,weight='length_meter',dtype=np.float16) # Use dtype >np.float16 for more precision
matrix

0          1          2          3          4          5     \
0      0.000000  19.477707  21.257921  19.482340  24.370579  22.620644   
1     16.907707   0.000000  10.998884   6.203304  13.171542  11.171609   
2     19.997921  10.858885   0.000000   6.273519  11.051758  14.601824   
3     15.502339   6.203304   6.273519   0.000000   9.506178  10.106242   
4     20.190578  11.051542  10.141758   8.366178   0.000000  14.794482   
...         ...        ...        ...        ...        ...        ...   
1253  26.107365  16.968328  13.988544  14.282963  14.861201  20.711267   
1254  18.705038   9.566002   8.396217   6.880637   9.608875  13.308942   
1255  16.365540   7.226502   9.786717   4.541137  10.339375  10.969441   
1256  16.715014   7.575979   6.406193   4.890613   7.088852  11.318917   
1257  19.041740   9.902705  10.982920   7.217339   8.495578  13.645644   

           6          7          8          9     ...       1248       1249  \
0     48.959641  21.764475  22.554434  19.405178  ...  23.824213  14.421368   
1     39.310604  12.825438  12.905397   8.206142  ...  12.375177   7.542331   
2     41.960819  16.255653  15.805614   6.086357  ...  15.805391  10.632546   
3     37.715240  11.760073  11.310033   4.540777  ...  11.309811   6.136965   
4     41.133480  16.448311  15.678271   7.159016  ...  15.998051  10.825205   
...         ...        ...        ...        ...  ...        ...        ...   
1253  47.690262  22.365097  21.915058  12.675801  ...  21.914837  16.741991   
1254  39.317940  14.962771  13.862731   4.503475  ...  14.512510   9.339664   
1255  36.278435  12.623271  10.293231   5.713975  ...  12.173010   7.000164   
1256  36.487911  12.972747  11.032707   2.513451  ...  12.522487   7.349640   
1257  39.314640  15.299474  14.849434   7.040178  ...  14.849213   9.676366   

           1250       1251       1252       1253       1254       1255  \
0     20.024340  20.100258  20.032118  27.937365  22.015039  20.085539   
1      9.765304   9.151222  10.383081  16.738329  10.816002   8.606503   
2      2.645518  12.581437  13.283297  13.978543   8.396217  10.486717   
3      7.119937   8.085856   8.787716  13.072963   7.150637   5.531137   
4     12.278176  12.774096  12.405954  15.361201   7.818875   9.999375   
...         ...        ...        ...        ...        ...        ...   
1253  13.424962  18.690882  19.232740   0.000000  11.675661  16.596161   
1254   9.032636  11.288555  10.590414  11.675661   0.000000   8.183835   
1255  10.853136   8.949056   7.350914  14.916162   7.613835   0.000000   
1256   8.542612   9.298532   7.760390  12.485638   3.173311   5.353811   
1257  13.119339  11.625257  12.327117  15.462364   7.890038   9.530538   

           1256       1257  
0     19.885014  22.431742  
1      8.685979  11.232705  
2      6.406193   9.602920  
3      5.020613   7.567339  
4      4.988852   8.575578  
...         ...        ...  
1253  11.815638  14.892365  
1254   3.173311   7.360038  
1255   5.483811   8.510538  
1256   0.000000   5.230014  
1257   5.060014   0.000000  

[1258 rows x 1258 columns]